# Using GeMMM

In this notebook we demonstrate how GeMMM can be used to sample origin-destination matrices for a specific set of MSOAs.

A table of available MSOAs is provided in the `tables` module. This includes additional information such as the Local Authority District (LAD), region and country that may be relevant when journey numbers are required for a larger area. For MSOAs in Wales and Scotland, the region and LAD columns are equivalent.

In [1]:
from gemmm.tables import gb_msoas
gb_msoas.head()

,msoa,msoa_name,lad,lad_name,region,region_name,country
0,E02000001,City of London 001,E09000001,City of London,E12000007,London,England
1,E02000002,Barking and Dagenham 001,E09000002,Barking and Dagenham,E12000007,London,England
2,E02000003,Barking and Dagenham 002,E09000002,Barking and Dagenham,E12000007,London,England
3,E02000004,Barking and Dagenham 003,E09000002,Barking and Dagenham,E12000007,London,England
4,E02000005,Barking and Dagenham 004,E09000002,Barking and Dagenham,E12000007,London,England


<br>
Suppose that we are interested in generating journey numbers for MSOAs located within the Cambridge LAD.

We can pull out the codes of MSOAs that satisfy this condition from the previous table.

In [2]:
LAD_NAME = 'Cambridge'
msoas = gb_msoas.query('lad_name==@LAD_NAME').msoa.to_numpy()

To sample journey numbers, we first provide the `OriginDestination` class with this array of MSOAs and a day type, either weekday or weekend. 

When running this for the first time, GeMMM will download [data files](https://github.com/ukhsa-collaboration/Gemmm/tree/main/model_data/) required by the Fourier series and radiation models for each day type. These are cached for future use.

In [3]:
from gemmm import OriginDestination
sampler = OriginDestination(msoas=msoas, day_type='weekday')

With this, we can now sample journey numbers for a single hour of the day or list of hours. Here, `n_realizations` specifies the number of realizations that will be generated for each hour. 

In [4]:
sampler.generate_sample(hours=[8, 12, 16], n_realizations=5)

Samples are stored as a dictionary in `X.samples` with keys given by the tuple *(x, y)*, where *x* is the hour and *y* is the realization number. Note that the numbering of realizations begins with zero.

Since the origin-destination matrices contain a high proportion of zeros when considering MSOAs over a large area, the generated matrices are stored in sparse matrix format. Specifically, the COOrdinate format is used, where the row attribute contains the indices of the start MSOAs, the col attribute contains the indices of the end MSOA, and the data attribute contains the number of journeys. The indices of the MSOAs refer to their position in the list originally provided to the `OriginDestination` class.

To present the output in a more readable format, the matrices can be converted to pandas DataFrames.

In [5]:
sampler.to_pandas(hour=8, realization=0, wide=True)

journeys                                                    \
end_msoa   E02003719 E02003720 E02003721 E02003722 E02003723 E02003724   
start_msoa                                                               
E02003719      216.0       1.0       4.0       3.0       2.0       9.0   
E02003720        3.0     266.0      11.0       3.0       8.0      10.0   
E02003721        2.0       5.0     323.0       3.0       8.0       5.0   
E02003722        4.0       5.0       1.0     426.0       0.0       6.0   
E02003723        8.0       6.0       3.0       2.0     434.0       8.0   
E02003724        9.0       8.0       9.0       1.0       6.0     500.0   
E02003725       13.0      12.0      21.0      10.0      12.0      31.0   
E02003726        3.0       5.0       5.0       0.0       1.0       4.0   
E02003727        2.0       1.0       5.0       1.0       8.0       4.0   
E02003728        5.0       7.0      19.0       8.0      19.0      13.0   
E02003729       13.0       5.0      24.0      10.0       5.0       8.0   
E02003730        7.0      21.0      25.0      13.0      25.0      17.0   
E02003731       12.0       6.0      12.0       8.0       9.0      14.0   

                                                                        \
end_msoa   E02003725 E02003726 E02003727 E02003728 E02003729 E02003730   
start_msoa                                                               
E02003719       36.0       6.0       4.0      22.0       4.0      32.0   
E02003720       41.0      12.0       2.0      25.0      11.0      59.0   
E02003721       43.0       6.0       3.0       7.0       6.0      37.0   
E02003722       21.0       3.0       1.0      11.0       7.0      24.0   
E02003723       22.0       3.0       2.0      13.0       2.0      27.0   
E02003724       78.0       6.0       4.0      22.0      15.0      62.0   
E02003725     1047.0      10.0      14.0      41.0      17.0      44.0   
E02003726       19.0     374.0       0.0       7.0       5.0      18.0   
E02003727       25.0       0.0     358.0       3.0       7.0      23.0   
E02003728       64.0       3.0       4.0     378.0       8.0      31.0   
E02003729       79.0       3.0       6.0      10.0     327.0      29.0   
E02003730       82.0      13.0      10.0      31.0      32.0     780.0   
E02003731       27.0       2.0       7.0      10.0       6.0      30.0   

                      
end_msoa   E02003731  
start_msoa            
E02003719       15.0  
E02003720       23.0  
E02003721       27.0  
E02003722       27.0  
E02003723       15.0  
E02003724       40.0  
E02003725       27.0  
E02003726       16.0  
E02003727        7.0  
E02003728       11.0  
E02003729        9.0  
E02003730       33.0  
E02003731      520.0

<br>
Sometimes it may be useful to know average journey numbers rather than sampled values. These averages can be extracted from the Fourier series and radiation model data for a specific hour.

In [6]:
import numpy as np

hour = 16
fourier_mean = np.zeros((len(sampler.msoas),)*2)
fourier_mean[sampler.fourier_data.row, sampler.fourier_data.col] = sampler.fourier_data.mean[hour]

radiation_mean = sampler.radiation_data.theta[hour] * sampler.radiation_data.mean

model_mean = fourier_mean + radiation_mean

## Saving and loading samples
In some instances, it is convenient to save the generated matrices for use later. In GeMMM, this can be achieved using the `save_sample` argument when generating the samples. If True, the samples will be saved in the current directory, otherwise the path to a specific directory must be provided. The filename is automatically set using a timestamp and the day type.

In [8]:
save_file = sampler.generate_sample(hours=[8, 12, 16], n_realizations=5, save_sample=True)

Saving samples to weekday_samples_2025-04-24--14-52-28.nc


To load the samples, we again use the `OriginDestination` class, but this time provide the full file path.

In [10]:
loader = OriginDestination(file=save_file)

Available hours: 8, 12, 16
Number of realizations: 5


We can then load a matrix for one of the available hours. If a realization is not specified, one will be selected at random.

In [11]:
loader.load_sample(hour=16, realization=0, wide=True)

journeys                                                    \
end_msoa   E02003719 E02003720 E02003721 E02003722 E02003723 E02003724   
start_msoa                                                               
E02003719      220.0       4.0       2.0       1.0       5.0      16.0   
E02003720        1.0     269.0      10.0       3.0       7.0       8.0   
E02003721        6.0      11.0     281.0       4.0       3.0       8.0   
E02003722        7.0       2.0       4.0     319.0       2.0       1.0   
E02003723        3.0       6.0       6.0       1.0     434.0      12.0   
E02003724       11.0      17.0      13.0      12.0       8.0     378.0   
E02003725       42.0      52.0      41.0      36.0      30.0      70.0   
E02003726       15.0      11.0      16.0       6.0       5.0       3.0   
E02003727        3.0       9.0       3.0       1.0       0.0       9.0   
E02003728       26.0      21.0       8.0      10.0      14.0      25.0   
E02003729       11.0      16.0       7.0       8.0       4.0      17.0   
E02003730       27.0      28.0      37.0      13.0      28.0      42.0   
E02003731       19.0      32.0      23.0      16.0      16.0      38.0   

                                                                        \
end_msoa   E02003725 E02003726 E02003727 E02003728 E02003729 E02003730   
start_msoa                                                               
E02003719       22.0       1.0       5.0      10.0       5.0      29.0   
E02003720       26.0       8.0       6.0      12.0       2.0      13.0   
E02003721       21.0       7.0       8.0      21.0      15.0      21.0   
E02003722        7.0       8.0       1.0      12.0      12.0       9.0   
E02003723       22.0       6.0       7.0       6.0       8.0      17.0   
E02003724       24.0       7.0       8.0      15.0       8.0      29.0   
E02003725     1158.0      51.0      41.0      67.0      54.0     138.0   
E02003726       28.0     257.0       2.0       8.0       7.0      18.0   
E02003727        7.0       3.0     314.0       6.0       6.0      18.0   
E02003728       50.0       6.0       7.0     359.0       7.0      28.0   
E02003729       19.0       4.0       3.0       7.0     266.0      24.0   
E02003730      112.0       9.0       9.0      32.0      26.0     752.0   
E02003731       42.0       6.0       8.0      20.0      15.0      36.0   

                      
end_msoa   E02003731  
start_msoa            
E02003719        6.0  
E02003720        3.0  
E02003721        9.0  
E02003722        8.0  
E02003723       10.0  
E02003724       11.0  
E02003725       36.0  
E02003726        9.0  
E02003727       11.0  
E02003728       10.0  
E02003729        7.0  
E02003730       17.0  
E02003731      494.0